## Import ##

In [1]:
!pip install biodivine_aeon==1.3.0a3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 20.3 MB/s eta 0:00:00


In [2]:
from biodivine_aeon import *
import requests

## Functions ##

In [3]:
class EnrichmentBehaviourClass:
    def __init__(self, ) -> None:
        self.attractors = list()
        self.attractor_types = list()

    def add_attractor(self, attractor):
        self.attractors.append(attractor)
        self.attractor_types.append(attractor.attractor_type)

    def goterm_intersection(self):
        intersect = self.attractors[0].go_terms_set
        for attractor in self.attractors[1:]:
            intersect = intersect.intersection(attractor.go_terms_set)
        return intersect

    def goterm_unique(self):
        unique = []
        for attractor in self.attractors:
            unique_set = attractor.go_terms_set
            for attractor2 in self.attractors:
                if attractor == attractor2: continue
                unique_set = unique_set.difference(attractor2.go_terms_set)
            unique.append(unique_set)
        return unique

    def __str__(self):
        result = "Behaviour class \n"
        for attractor in self.attractors:
            result += "|-- " + str(attractor) + "\n"
        return result

    def __repr__(self):
        result = "Behaviour class \n"
        for attractor in self.attractors:
            result += "|-- " + str(attractor) + "\n"
        return result

class EnrichmentAttractor:
    def __init__(self, attractor_type, enrichment_result, fdr) -> None:
        self.fdr = fdr
        self.attractor_type = attractor_type
        self.goterms = dict()
        self.go_terms_set = set()
        self.mapped_ids = enrichment_result.mapped_ids
        self.unmapped_ids = enrichment_result.unmapped_ids

        for process in enrichment_result.result:
            go_term = EnrichmentGOterm(process)
            if go_term.fdr > self.fdr: continue
            self.goterms[go_term.go_id] = go_term
            self.go_terms_set.add(go_term.go_id)

    def get_goterms_by_set(self, wanted):
        return [self.goterms[go_id] for go_id in wanted if go_id in self.goterms]

    def get_all_goterms(self):
        return self.goterms

    def get_plus_goterms(self):
        return [goterm for goterm in self.goterms.values() if goterm.plus_minus == "+"]

    def get_minus_goterms(self):
        return [goterm for goterm in self.goterms.values() if goterm.plus_minus == "-"]

    def __str__(self):
        return f"{self.attractor_type}"

    def __repr__(self):
        return f"{self.attractor_type}"

class EnrichmentGOterm:
    def __init__(self, process) -> None:
        self.go_id = process.get("term", {}).get("id", "")
        self.process_name = process["term"]["label"]

        self.fold_enrichment = process["fold_enrichment"]
        self.fdr = process["fdr"]
        self.expected = process["expected"]
        self.number_in_reference = process["number_in_reference"]
        self.p_value = process["pValue"]
        self.plus_minus = process["plus_minus"]

    def __repr__(self):
        return f"{self.plus_minus}{self.process_name}"

    def __str__(self):
        return f"{self.plus_minus}{self.process_name}"


In [4]:
import requests


class EnrichmentResult:
    def __init__(self, enrichmentData):
        self.data = enrichmentData

        self.input = self.data["results"]["input_list"]
        self.organism = self.input["organism"]
        self.mapped_ids = self.input["mapped_ids"]
        self.mapped_count = self.input["mapped_count"]
        self.unmapped_ids = self.input["unmapped_ids"]
        self.unmapped_count = self.input["unmapped_count"]
        self.result = self.data["results"]["result"]  # Sorted by FDR


def prepare_list_for_enrichment(nodes):
    as_string = str(nodes)[1:-1]
    as_string = as_string.replace("\'", "")
    return as_string


def prepare_enrichment_result(enrichment):
    if isinstance(enrichment, dict) and 'search' in enrichment and isinstance(enrichment['search'], dict) and 'error' in enrichment['search']:
        return None

    enrichment_result = EnrichmentResult(enrichment)
    return enrichment_result


def get_enrichment(genes_string, organism_id, goterm_type, test_type="FISHER", correction="FDR"):
    input_genes = genes_string
    organism = organism_id
    test_type = test_type   # FISHER, BINOMIAL
    correction = correction # FDR, BONFERRONI, NONE
    # refInputList  <- potential extension
    # refOrganism

    match goterm_type:
        case "MF": data_type = "GO:0003674"
        case "BP": data_type = "GO:0008150"
        case "CC": data_type = "GO:0005575"
        case _:
            print('Wrong goterm_type. Use "MF","BP","CC" instead. Ending get_enrichment function.')
            return

    req_link = f"https://pantherdb.org/services/oai/pantherdb/enrich/overrep?geneInputList={input_genes}&organism={organism}&annotDataSet={data_type}&enrichmentTestType={test_type}&correction={correction}"
    headers = {"Content-Type": "application/json"}
    response = requests.get(req_link, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data

    print("Failed to get data. Ending get_enrichment function.")
    return


In [5]:
def make_union(attractors):
    unionized = attractors[0]
    for attractor in attractors[1:]:
        unionized = unionized.union(attractor)
    return unionized


def get_stability_percentage(node, stg, attractor):
    variable_on = stg.mk_subspace({node: True})

    on_in_attractor = attractor.intersect(variable_on).vertices().cardinality()
    off_in_attractor = attractor.minus(variable_on).vertices().cardinality()

    return round((on_in_attractor / (on_in_attractor + off_in_attractor)) * 100.0, 2)


def get_stabilities(classifiers, attractors, stg, calculate_unstable=True):
    all_dict = dict()
    unionized_attractors = make_union(attractors)
    for classifier in classifiers:
        subset = unionized_attractors.intersect_colors(classifiers[classifier])
        enclosing = subset.vertices().enclosing_named_subspace()
        unstable_variables = dict()

        if calculate_unstable:
            all_variables = set(stg.network_variable_names())
            stable_variables = set(enclosing)
            to_calculate = all_variables.difference(stable_variables)

            for variable in to_calculate:
                unstable_variables[variable] = get_stability_percentage(variable, stg, subset)

        stability_dict = dict()
        for node in enclosing:
            if enclosing[node] == True:
                stability_dict[node] = 100
            else:
                stability_dict[node] = 0
        if calculate_unstable:
            stability_dict = stability_dict | unstable_variables

        all_dict[classifier] = stability_dict
    return all_dict


def get_stable_nodes(classifiers, attractors, stg, lower_bound=0, upper_bound=100):
    calculate_unstable = True
    if lower_bound == 100 or upper_bound == 0:
        calculate_unstable = False

    stabilities = get_stabilities(classifiers, attractors, stg, calculate_unstable)

    result_dict = dict()
    for one_class in stabilities:
        print(one_class)
        print("  |", end='')

        sub = []
        for node in stabilities[one_class]:
            if stabilities[one_class][node] > upper_bound:
                continue
            if stabilities[one_class][node] < lower_bound:
                continue

            print(node + ": " + str(stabilities[one_class][node]) + "|", end='')
            sub.append(node)

        print()
        result_dict[one_class] = sub
    return result_dict


In [6]:
def get_evaluated_nodes(phenotype, evaluation=True):
    result_list = []
    for node in phenotype:
        if node[0] == "+" and evaluation:
            result_list.append(node[1:])
        elif node[0] == "-" and not evaluation:
            result_list.append(node[1:])
    return result_list


## Pipelines ##

### Network preprocessing ###

In [7]:
network = BooleanNetwork.from_file("deathReceptorSignaling-TTT.aeon")
print(network)

BooleanNetwork(variables=28, regulations=45, explicit_parameters=0, implicit_parameters=0)


In [8]:
ctx = SymbolicSpaceContext(network)
stg = AsynchronousGraph(network, ctx)

In [9]:
classification = Classification.classify_stable_phenotypes(ctx, stg) # Used by Pipeline1

In [ ]:
attractors = Attractors.attractors(stg)
attractorClassifs = Classification.classify_attractor_bifurcation(stg, attractors) # Used by Pipeline2
attractorClassifs

{Class(["stability", "stability", "stability"]): ColorSet(cardinality=1, symbolic_size=2)}

In [ ]:
attractors_types = list(attractorClassifs)[0].feature_list()

### Pipeline 1 ###

Extraction of evaluated nodes from attractor classes

In [ ]:
class_results = []
for res in range(len(classification)):
  a = list(classification)[res]
  class_results.append(a.feature_list())

In [ ]:
for i in class_results:
  print(i)

['+ATP', '+BAX', '+CASP3', '+CASP8', '+Cyt_c', '+DISC_FAS', '+DISC_TNF', '+FADD', '+FASL', '+MOMP', '+SMAC', '+TNF', '+TNFR', '+apoptosis', '+apoptosome', '-BCL2', '-IKK', '-MPT', '-NFkB', '-NonACD', '-RIP1', '-RIP1k', '-RIP1ub', '-ROS', '-XIAP', '-cFLIP', '-cIAP', '-survival']
['+BAX', '+CASP8', '+Cyt_c', '+DISC_FAS', '+DISC_TNF', '+FADD', '+FASL', '+MOMP', '+MPT', '+NonACD', '+ROS', '+SMAC', '+TNF', '+TNFR', '-ATP', '-BCL2', '-CASP3', '-IKK', '-NFkB', '-RIP1', '-RIP1k', '-RIP1ub', '-XIAP', '-apoptosis', '-apoptosome', '-cFLIP', '-cIAP', '-survival']
['+ATP', '+BCL2', '+DISC_FAS', '+DISC_TNF', '+FADD', '+FASL', '+IKK', '+NFkB', '+RIP1', '+RIP1k', '+RIP1ub', '+TNF', '+TNFR', '+XIAP', '+cFLIP', '+cIAP', '+survival', '-BAX', '-CASP3', '-CASP8', '-Cyt_c', '-MOMP', '-MPT', '-NonACD', '-ROS', '-SMAC', '-apoptosis', '-apoptosome']


Iteration over attractors

In [ ]:
behaviour_class = EnrichmentBehaviourClass()

for nodes, attractor_type in zip(class_results, attractors_types):
    to_enrich = get_evaluated_nodes(nodes, True) # select positively evaluated nodes
    to_enrich = prepare_list_for_enrichment(to_enrich) # Transform list into string that can be used in url
    enrichment = get_enrichment(to_enrich, "9606", "BP") # Use PANTHER API
    enrichment_result = prepare_enrichment_result(enrichment) # Process the results from API
    calculated_attractor = EnrichmentAttractor(attractor_type, enrichment_result, 0.05) # Create class where the results are stored for given attractor
    behaviour_class.add_attractor(calculated_attractor) # All attractors are under single behaviour class


Attractors contained in a given behaviour class

In [ ]:
behaviour_class

Behaviour class 
|-- stability
|-- stability
|-- stability

In [ ]:
behaviour_class.attractors

[stability, stability, stability]

Full description of GO Terms

In [ ]:
behaviour_class.attractors[0].get_all_goterms()

{'GO:0008625': +extrinsic apoptotic signaling pathway via death domain receptors,
 'GO:0043525': +positive regulation of neuron apoptotic process,
 'GO:0097190': +apoptotic signaling pathway,
 'GO:0097191': +extrinsic apoptotic signaling pathway,
 'GO:0097300': +programmed necrotic cell death,
 'GO:0043065': +positive regulation of apoptotic process,
 'GO:0043068': +positive regulation of programmed cell death,
 'GO:0097527': +necroptotic signaling pathway,
 'GO:0043523': +regulation of neuron apoptotic process,
 'GO:0071214': +cellular response to abiotic stimulus,
 'GO:0104004': +cellular response to environmental stimulus,
 'GO:0070266': +necroptotic process,
 'GO:0006915': +apoptotic process,
 'GO:0012501': +programmed cell death,
 'GO:0008219': +cell death,
 'GO:2001235': +positive regulation of apoptotic signaling pathway,
 'GO:0051707': +response to other organism,
 'GO:0043207': +response to external biotic stimulus,
 'GO:0042981': +regulation of apoptotic process,
 'GO:0009607

Set operations over attractor's GO Terms

In [ ]:
set0 = behaviour_class.attractors[0].go_terms_set
set1 = behaviour_class.attractors[1].go_terms_set
set2 = behaviour_class.attractors[2].go_terms_set

In [ ]:
sets_intersection = set0.intersection(set1).intersection(set2)
sets_intersection

{'GO:0001775',
 'GO:0001776',
 'GO:0001819',
 'GO:0002237',
 'GO:0002260',
 'GO:0002376',
 'GO:0002521',
 'GO:0002684',
 'GO:0002761',
 'GO:0002763',
 'GO:0002831',
 'GO:0006915',
 'GO:0006952',
 'GO:0007166',
 'GO:0008219',
 'GO:0008625',
 'GO:0008630',
 'GO:0009605',
 'GO:0009607',
 'GO:0009615',
 'GO:0009966',
 'GO:0009967',
 'GO:0010623',
 'GO:0010646',
 'GO:0010647',
 'GO:0010720',
 'GO:0012501',
 'GO:0016525',
 'GO:0022612',
 'GO:0023051',
 'GO:0023056',
 'GO:0030097',
 'GO:0030335',
 'GO:0032496',
 'GO:0032677',
 'GO:0032729',
 'GO:0032757',
 'GO:0034341',
 'GO:0038034',
 'GO:0040017',
 'GO:0042981',
 'GO:0043029',
 'GO:0043065',
 'GO:0043067',
 'GO:0043068',
 'GO:0043069',
 'GO:0043122',
 'GO:0043123',
 'GO:0043207',
 'GO:0043523',
 'GO:0043525',
 'GO:0044419',
 'GO:0045088',
 'GO:0045321',
 'GO:0045639',
 'GO:0045994',
 'GO:0046666',
 'GO:0048583',
 'GO:0048584',
 'GO:0048732',
 'GO:0050776',
 'GO:0051402',
 'GO:0051707',
 'GO:0061048',
 'GO:0070227',
 'GO:0070228',
 'GO:00702

In [ ]:
behaviour_class.attractors[0].get_goterms_by_set(sets_intersection)

[+apoptotic signaling pathway,
 +regulation of response to stress,
 +positive regulation of apoptotic process,
 +hemopoiesis,
 +positive regulation of programmed cell death,
 +programmed cell death involved in cell development,
 +positive regulation of canonical NF-kappaB signal transduction,
 +cell activation,
 +response to Gram-negative bacterium,
 +regulation of innate immune response,
 +apoptotic process,
 +positive regulation of interleukin-8 production,
 +regulation of response to biotic stimulus,
 +regulation of neuron apoptotic process,
 +regulation of interleukin-8 production,
 +positive regulation of cell development,
 +cell surface receptor signaling pathway,
 +regulation of response to stimulus,
 +positive regulation of neuron apoptotic process,
 +T cell homeostasis,
 +regulation of extrinsic apoptotic signaling pathway,
 +leukocyte apoptotic process,
 +response to virus,
 +negative regulation of vasculature development,
 +regulation of signal transduction,
 +regulation of 

### Pipeline2 ###

Last two parameters of get_stable_nodes() describe lower and upper bound.

In [ ]:
enrichment_input_pipeline2 = get_stable_nodes(attractorClassifs, attractors, stg, 50, 70)

["stability", "stability", "stability"]
  |Cyt_c: 66.67|BAX: 66.67|ATP: 66.67|MOMP: 66.67|CASP8: 66.67|SMAC: 66.67|


In [ ]:
behaviour_class_2 = EnrichmentBehaviourClass()

to_enrich2 = prepare_list_for_enrichment(behaviour_class_2)
enrichment2 = get_enrichment(to_enrich2, 9606, "BP")
enrichment_result2 = prepare_enrichment_result(enrichment2)
calculated_attractor2 = EnrichmentAttractor("combinedAttractor", enrichment_result, 0.05) # Custom name is given to the attractor
behaviour_class_2.add_attractor(calculated_attractor2)

In [ ]:
behaviour_class_2

Behaviour class 
|-- combinedAttractor

In [ ]:
behaviour_class_2.attractors[0].get_all_goterms()

{'GO:0008625': +extrinsic apoptotic signaling pathway via death domain receptors,
 'GO:0097527': +necroptotic signaling pathway,
 'GO:0097191': +extrinsic apoptotic signaling pathway,
 'GO:0070266': +necroptotic process,
 'GO:2001236': +regulation of extrinsic apoptotic signaling pathway,
 'GO:0097300': +programmed necrotic cell death,
 'GO:0043123': +positive regulation of canonical NF-kappaB signal transduction,
 'GO:0046666': +retinal cell programmed cell death,
 'GO:2001238': +positive regulation of extrinsic apoptotic signaling pathway,
 'GO:0043122': +regulation of canonical NF-kappaB signal transduction,
 'GO:0097190': +apoptotic signaling pathway,
 'GO:0006915': +apoptotic process,
 'GO:0098542': +defense response to other organism,
 'GO:0012501': +programmed cell death,
 'GO:0008219': +cell death,
 'GO:2001233': +regulation of apoptotic signaling pathway,
 'GO:0051707': +response to other organism,
 'GO:0043207': +response to external biotic stimulus,
 'GO:0051402': +neuron ap